# Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [1]:
# !pip install catboost

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
import catboost as catb
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

%matplotlib inline

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

### 1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

Возьмем Census Income Data Set

Abstract: Predict whether income exceeds $50K/yr based on census data. Also known as "Adult" dataset.

Data Set Information:

Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

Prediction task is to determine whether a person makes over 50K a year.

In [3]:
data = pd.read_csv("adult.data")
# data = pd.read_csv("/content/sample_data/adult.data")


data.columns = ['age', 'workclass', 'fnlwgt', 'education',
              'education-num', 'marital-status', 'occupation',
              'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
              'hours-per-week', 'native-country', 'target']
data.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K


In [4]:
print(data.shape)

(32560, 15)


Данные содержат 14 признаков и таргет в виде строки заработает ли человек более 50 000 или нет. Всего имееем 32560 записей.

Посмотрим на соотношение классов

In [5]:
data.iloc[:, -1].value_counts()

 <=50K    24719
 >50K      7841
Name: target, dtype: int64

### 2. сделать feature engineering

Разобъем данные на данные и таргет, а так же заменим строки на бинарные признаки и проверим соотношение, что ничего не изменилось.

In [6]:
data['target'] = data['target'].apply(lambda x: 0 if x == ' <=50K' else 1)

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

data.iloc[:, -1].value_counts()

0    24719
1     7841
Name: target, dtype: int64

Разобъем выборку на тренировочную и тестовую

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)

### 3. обучить любой классификатор (какой вам нравится)

In [8]:
cat_features = ['workclass', 
                'education', 
                'marital-status', 
                'occupation', 
                'relationship', 
                'race',
                'sex',
                'native-country']

model = catb.CatBoostClassifier(random_state=42, cat_features=cat_features, silent=True)

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

evaluate_results(y_test, y_predict)


Classification results:
f1: 72.20%
roc: 80.54%
recall: 67.13%
precision: 78.09%


### 4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [9]:
mod_data = data.copy()

# Возьмем позитивные индексы
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]

# перемешаем их
np.random.shuffle(pos_ind)

# оставим только 25% позитивных маркеров
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))

print(f'Используется {pos_sample_len}/{len(pos_ind)} как позитивные и неразмеченные')
pos_sample = pos_ind[:pos_sample_len]

Используется 1961/7841 как позитивные и неразмеченные


In [10]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    30599
 1     1961
Name: class_test, dtype: int64


In [11]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 5. применить random negative sampling для построения классификатора в новых условиях

In [12]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1961, 16) (1961, 16)


In [13]:
sample_train.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,class_test
30015,31,State-gov,440129,Some-college,10,Divorced,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,1,1
3841,55,Federal-gov,189985,Some-college,10,Divorced,Adm-clerical,Unmarried,Black,Female,0,0,40,United-States,0,-1
27244,43,Private,289669,Masters,14,Divorced,Prof-specialty,Unmarried,White,Female,0,2547,40,United-States,1,1


In [14]:
cat_cols = sample_train.select_dtypes(include='object').columns.tolist()

In [15]:
model = catb.CatBoostClassifier(random_state=42, cat_features=cat_cols, silent=True)

sample_data = sample_train.iloc[:,:-2]
sample_target = sample_train.iloc[:,-2]

model.fit(sample_data, 
          sample_target)

y_predict = model.predict(sample_test.iloc[:,:-2].values)

evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 61.96%
roc: 83.20%
recall: 90.64%
precision: 47.07%


Видим, что метрика f1 стала хуже, т.к. мы фактически при сэмплировании часть позиитивных классов отметили как негативные. Хотя метрика roc auc выросла. 

### 7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [16]:
# Обернем все предыдущие вычисления в функцию
def tst (data, p):
    mod_data = data.copy()

    # Возьмем позитивные индексы
    pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]

    # перемешаем их
    np.random.shuffle(pos_ind)

    pos_sample_len = int(np.ceil(p * len(pos_ind)))

    pos_sample = pos_ind[:pos_sample_len]

    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1

    x_data = mod_data.iloc[:,:-2].values # just the X 
    y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
    y_positive = mod_data.iloc[:,-2].values # original class

    mod_data = mod_data.sample(frac=1)
    neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
    sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
    pos_sample = mod_data[mod_data['class_test']==1]
    print(neg_sample.shape, pos_sample.shape)
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

    cat_cols = sample_train.select_dtypes(include='object').columns.tolist()

    model = catb.CatBoostClassifier(random_state=42, cat_features=cat_cols, silent=True)

    sample_data = sample_train.iloc[:,:-2]
    sample_target = sample_train.iloc[:,-2]

    model.fit(sample_data, 
              sample_target)

    y_predict = model.predict(sample_test.iloc[:,:-2].values)

    evaluate_results(sample_test.iloc[:,-2].values, y_predict)



Исследуем метрики на разных долях

In [17]:
list_p = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

for i in list_p:
    print(i)
    tst(data, i)
    print('============================\n')

0.1
(785, 16) (785, 16)
Classification results:
f1: 63.53%
roc: 81.86%
recall: 90.69%
precision: 48.89%

0.2
(1569, 16) (1569, 16)
Classification results:
f1: 62.77%
roc: 82.53%
recall: 89.19%
precision: 48.42%

0.3
(2353, 16) (2353, 16)
Classification results:
f1: 60.11%
roc: 83.15%
recall: 91.18%
precision: 44.84%

0.4
(3137, 16) (3137, 16)
Classification results:
f1: 57.24%
roc: 83.37%
recall: 90.44%
precision: 41.87%

0.5
(3921, 16) (3921, 16)
Classification results:
f1: 53.71%
roc: 83.90%
recall: 91.32%
precision: 38.04%

0.6
(4705, 16) (4705, 16)
Classification results:
f1: 49.44%
roc: 83.94%
recall: 89.32%
precision: 34.18%

0.7
(5489, 16) (5489, 16)
Classification results:
f1: 43.56%
roc: 83.79%
recall: 88.54%
precision: 28.88%

0.8
(6273, 16) (6273, 16)
Classification results:
f1: 35.56%
roc: 84.91%
recall: 89.23%
precision: 22.20%



При исследовании вероятностей видим, что чем больше сэмплируем, тем хуже становится метрика. Выходит что фактически мы сильнее "путаем" модель ложными данными при обычной классификации с нормальными данными. Но если бы фактически мы не знали четкого разбиения, то это нам помогло бы достичь точности в 61-63%.